In [1]:
import pyntcloud
import pickle
import tensorflow as tf
import numpy as np
import importlib
from train import provider
from train import box_util

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
importlib.reload(box_util)
importlib.reload(provider)

<module 'train.provider' from '/datasets/home/90/490/scgullap/ece_285_proj/frustum-pointnets/train/provider.py'>

In [3]:
NUM_POINT=2048
TRAIN_DATASET = provider.FrustumDataset(npoints=NUM_POINT, split='train',\
                            rotate_to_center=True, random_flip=False, random_shift=False, one_hot=True)

In [4]:
VAL_DATASET = provider.FrustumDataset(npoints=NUM_POINT, split='val',\
                            rotate_to_center=True, random_flip=False, random_shift=False, one_hot=True)

In [5]:
point_set,labels,c,d,e,f,g,h,i = TRAIN_DATASET[0]

In [6]:
len(TRAIN_DATASET)

73480

In [7]:
point_set.shape

(2048, 4)

In [8]:
TRAIN_DATASET.get_center_view_box3d(1)

array([[ 0.63603565,  1.47      ,  8.95986384],
       [ 0.7288103 ,  1.47      ,  8.48891494],
       [-0.44856195,  1.47      ,  8.25697831],
       [-0.5413366 ,  1.47      ,  8.72792721],
       [ 0.63603565, -0.42      ,  8.95986384],
       [ 0.7288103 , -0.42      ,  8.48891494],
       [-0.44856195, -0.42      ,  8.25697831],
       [-0.5413366 , -0.42      ,  8.72792721]])

In [9]:
def indices(window_len, stride_len, point_set, labels, corners_gt):
    IOU = []
    start = []
    end = []
    IOU_1 = []
    IOU_2 = []
    corner = []
    j = np.min(point_set,0)[2]
    i = j+window_len
    idtuple=set()
    while i<=np.max(point_set,0)[2]:
            max_ind = np.searchsorted(point_set[:,2], i)
            min_ind = np.searchsorted(point_set[:,2], j)
            if (min_ind,max_ind) not in idtuple and max_ind-min_ind>5:
                #print('fgbd')
                idtuple.add((min_ind,max_ind))
                labels_window = np.zeros_like(point_set[:,0])
                labels_window[min_ind:max_ind] = 1.0
                union = np.sum(np.logical_or(labels, labels_window))
                inter = np.sum(np.logical_and(labels, labels_window))
                if (inter*1.0/union) < 0.3 or (inter*1.0/union) >0.7 :
                    end.append(max_ind)
                    start.append(min_ind)
                    IOU.append(inter*1.0/union)
            j = j+stride_len
            i = j+window_len
    IOU = np.array(IOU)>=0.7
    return np.array(start),np.array(end), np.array(IOU)



In [10]:
# points in each window:2048
# number of windows
#

def get_list(start_idx,end_idx,num_points=1024,ensure_all=True):
    """
    param:
    start_idx  : list of indexes of the starting point in each window
    end_idx    : list of indees of ending point in each window
    NOTES: we used two lists because we ran into issues with tensorflow when we used tuples,
           this is a quick hack to avoid that
    num_points : number of points to be sampled from i.e number of points in each window
    ensure_all : in the case that there are fewwer than required points ensures all points are picked atleast once if selected as True
    
    
    returns:
    out:  an array of size 
          
          num_windowsxnum_pointsx1
          
          this will be used inside tensorflow to divide into windows 
          after we pass the points trough pointnet
    """
    
    out= []
    for i in range(np.shape(start_idx)[0]):
        
        if end_idx[i]-start_idx[i]+1<=num_points:
            indexes=np.arange(start_idx[i],end_idx[i])
            indexes2=np.random.choice(indexes, num_points-np.shape(indexes)[0],replace=True)
            indexes=np.concatenate((indexes,indexes2))
        else:
            indexes=np.random.choice(np.arange(start_idx[i],end_idx[i]),num_points,replace=False)
        out.append(np.reshape(indexes,(-1,1)))
    
    return np.array(out)


In [11]:
import tf_util #for conv2d
from model_util import NUM_HEADING_BIN, NUM_SIZE_CLUSTER, NUM_OBJECT_POINT
from model_util import point_cloud_masking, get_center_regression_net
from model_util import placeholder_inputs, parse_output_to_tensors, get_loss
from model_util import huber_loss

In [12]:
# https://github.com/ailias/Focal-Loss-implement-on-Tensorflow/blob/master/focal_loss.py


from tensorflow.python.ops import array_ops

def focal_loss(sigmoid_p, target_tensor, weights=None, alpha=0.99, gamma=2):
    r"""Compute focal loss for predictions.
        Multi-labels Focal loss formula:
            FL = -alpha * (z-p)^gamma * log(p) -(1-alpha) * p^gamma * log
            (1-p)
                 ,which alpha = 0.25, gamma = 2, p = sigmoid(x), z = target_tensor.
    Args:
     prediction_tensor: A float tensor of shape [batch_size, num_anchors,
        num_classes] representing the predicted logits for each class
     target_tensor: A float tensor of shape [batch_size, num_anchors,
        num_classes] representing one-hot encoded classification targets
     weights: A float tensor of shape [batch_size, num_anchors]
     alpha: A scalar tensor for focal loss alpha hyper-parameter
     gamma: A scalar tensor for focal loss gamma hyper-parameter
    Returns:
        loss: A (scalar) tensor representing the value of the loss function
    """
    #sigmoid_p = tf.nn.sigmoid(prediction_tensor)
    zeros = array_ops.zeros_like(target_tensor, dtype=target_tensor.dtype)
    
    # For poitive prediction, only need consider front part loss, back part is 0;
    # target_tensor > zeros <=> z=1, so poitive coefficient = z - p.
    pos_p_sub = array_ops.where(target_tensor > zeros, target_tensor - sigmoid_p, zeros)
    
    # For negative prediction, only need consider back part loss, front part is 0;
    # target_tensor > zeros <=> z=1, so negative coefficient = 0.
    neg_p_sub = array_ops.where(target_tensor > zeros, zeros, sigmoid_p)
    per_entry_cross_ent = - alpha * (pos_p_sub ** gamma) * tf.log(tf.clip_by_value(sigmoid_p, 1e-8, 1.0)) \
                          - (1.0 - alpha) * (neg_p_sub ** gamma) * tf.log(tf.clip_by_value(1.0 - sigmoid_p, 1e-8, 1.0))
    return tf.reduce_sum(per_entry_cross_ent)

In [13]:
# def classification_net(point_cloud, label_out_5, label_out_1, label_out_2, label_out_3, label_out_4, idx5, idx1, idx2, idx3, idx4, 
#                             is_training=True, bn_decay=True):
def classification_net(point_cloud, label_out_5,  idx5, 
                            is_training=True, bn_decay=True):
    '''
    3D PointNet network.
    param:
    
    point_cloud: TF tensor in shape (1,N,4)
                 frustum point clouds with XYZ and intensity in point channels
                 XYZs are in frustum coordinate
    label_out: TF tensor in shape (1,1)
            length-1 vector indicating the classification as car or background
    
    ??????????????????????????????????????????????????????????????????????????????????????
    
    is_training: TF boolean scalar
    bn_decay: TF float scalar
    end_points: dict                 SHOULD CHANGE THIS
    Output:                                         
    logits: TF tensor in shape (B,N,2), scores for bkg/clutter and object
    end_points: dict
    ??????????????????????????????????????????????????????????????????????????????????????
    '''
    
    batch_size = point_cloud.get_shape()[0].value #1 in our case
    num_point = point_cloud.get_shape()[1].value  #N in our case
  
    net = tf.expand_dims(point_cloud, 2)
    #print(net.get_shape())

    net = tf_util.conv2d(net, 64, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv1', bn_decay=bn_decay)
    net = tf_util.conv2d(net, 64, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv2', bn_decay=bn_decay)
    point_feat = tf_util.conv2d(net, 64, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv3', bn_decay=bn_decay)
    net = tf_util.conv2d(point_feat, 128, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv4', bn_decay=bn_decay)
    features = tf_util.conv2d(net, 1024, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv5', bn_decay=bn_decay)
    
    features=tf.squeeze(features,axis=2)
    features=tf.squeeze(features,axis=0)   

    #features=tf.placeholder(tf.float32,[1,1024,4])

    def fn(x):
        return tf.gather_nd(features,x)
    
    windows_5 =tf.map_fn(fn,idx5,dtype=tf.float32)
#     windows_1 = tf.map_fn(fn,idx1,dtype=tf.float32)
#     windows_2 = tf.map_fn(fn,idx2,dtype=tf.float32)
#     windows_3 = tf.map_fn(fn,idx3,dtype=tf.float32)
#     windows_4 = tf.map_fn(fn,idx4,dtype=tf.float32)
    
    windows_5_1024 = tf.reduce_max(windows_5,axis=1)
#     windows_1_1024 = tf.reduce_max(windows_1,axis=1)
#     windows_2_1024 = tf.reduce_max(windows_2,axis=1)
#     windows_3_1024 = tf.reduce_max(windows_3,axis=1)
#     windows_4_1024 = tf.reduce_max(windows_4,axis=1)
    
    
    winex_5 =  tf.reshape(windows_5_1024,shape=[-1,1,1,1024])
#     winex_1 =  tf.reshape(windows_1_1024,shape=[-1,1,1,1024])
#     winex_2 =  tf.reshape(windows_2_1024,shape=[-1,1,1,1024])
#     winex_3 =  tf.reshape(windows_3_1024,shape=[-1,1,1,1024])
#     winex_4 =  tf.reshape(windows_4_1024,shape=[-1,1,1,1024])
   
    winex = tf.concat([winex_5], axis=0)
    label_out = tf.concat([label_out_5],axis=0)

    
    
#     winex = tf.concat([winex_5, winex_1, winex_2, winex_3, winex_4], axis=0)
#     label_out = tf.concat([label_out_5,label_out_1, label_out_2, label_out_3, label_out_4],axis=0)

    #classification head
    d1 = tf_util.conv2d(winex, 512, [1,1],
                             padding='VALID', stride=[1,1],
                             bn=True, is_training=is_training,
                             scope='dense1', bn_decay=bn_decay)
    d2 = tf_util.conv2d(d1, 256, [1,1],
                             padding='VALID', stride=[1,1],
                             bn=True, is_training=is_training,
                             scope='dense2', bn_decay=bn_decay)
    d2=tf.layers.dropout(d2,rate=0.5, training=is_training, name='drop1')
    
    logits = tf_util.conv2d(d2, 1, [1,1],
                             padding='VALID', stride=[1,1],
                             bn=True, is_training=is_training,
                             scope='out', bn_decay=bn_decay, activation_fn=None)
    
    logits = tf.squeeze(logits)
    logits=tf.reshape(tf.squeeze(logits),(1,-1))
    sig = tf.sigmoid(logits)

    #sig = tf.reshape(tf.squeeze(sig),(-1,1))
    y=tf.reshape(sig,[-1])
    
    max_ind = tf.argmax(y)
    
    tp = tf.equal(label_out[max_ind], 1)
    
    
    differentiable_round = tf.maximum(y-0.499,0)
    differentiable_round = differentiable_round * 10000
    y_pred = tf.minimum(differentiable_round, 1)
    #y_pred=tf.round(y)
    
    true_pos  = tf.reduce_sum(tf.cast(tf.logical_and(tf.cast(label_out,tf.bool),tf.cast(y_pred,tf.bool)),tf.float32))
    
    false_pos = tf.reduce_sum(tf.cast(tf.logical_and(tf.logical_not(tf.cast(label_out,tf.bool)),tf.cast(y_pred,tf.bool)),tf.float32))
    true_neg  = tf.reduce_sum(tf.cast(tf.logical_not(tf.logical_or(tf.cast(label_out, tf.bool),\
                                                                   tf.cast(y_pred, tf.bool))),tf.float32))
    false_neg = tf.reduce_sum(tf.cast(tf.logical_and(tf.logical_not\
                                                     (tf.cast(y_pred,tf.bool)),tf.cast(label_out,tf.bool)),tf.float32))
    return sig,logits, label_out,true_pos,false_pos,true_neg,false_neg, tp, max_ind

In [14]:
# inputs=tf.placeholder(tf.float32,[None,2048,4])
# idx_5 =tf.placeholder(tf.int32,[None,num_points_5,1])
# is_training = tf.placeholder(tf.bool)
# label_out_5 = tf.placeholder(tf.int32, [None])

In [15]:
tf.reset_default_graph()
with tf.device('/device:GPU:0'):
    num_points_5 =512
#     num_points_1 = 512
#     num_points_2 = 512
#     num_points_3 = 512
#     num_points_4 = 512
    
    inputs=tf.placeholder(tf.float32,[None,2048,4])#put placeholder

    idx_5 =tf.placeholder(tf.int32,[None,num_points_5,1])

#     idx_1 = tf.placeholder(tf.int32,[None,num_points_1,1])
    
#     idx_2 = tf.placeholder(tf.int32,[None,num_points_2,1])

#     idx_3 = tf.placeholder(tf.int32,[None,num_points_3,1])
    
#     idx_4 = tf.placeholder(tf.int32,[None,num_points_4,1])

    is_training = tf.placeholder(tf.bool)
    

    label_out_5 = tf.placeholder(tf.int32, [None])
#     label_out_1 = tf.placeholder(tf.int32, [None])
#     label_out_2 = tf.placeholder(tf.int32, [None])
#     label_out_3 = tf.placeholder(tf.int32, [None])
#     label_out_4 = tf.placeholder(tf.int32, [None])

    sig,logits, label_out,tp,fp,tn,fn, tp_num, maxi = classification_net(inputs, label_out_5, \
                                         idx_5, is_training)
    
    
#     sig,logits, label_out,tp,fp,tn,fn = classification_net(inputs, label_out_5, label_out_1,label_out_2, label_out_3, label_out_4, \
#                                          idx_5, idx_1, idx_2, idx_3, idx_4, is_training)
    
    func_fl = focal_loss(tf.squeeze(sig),tf.squeeze(tf.cast(label_out, tf.float32)), gamma=0.75, alpha=0.25)
    print(label_out.get_shape())
#     loss=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
#     labels=tf.reshape(tf.cast(label_out,tf.float32),(-1,1)),
#     logits=tf.reshape(logits,(-1,1))))

    loss = tf.reduce_mean(func_fl)
#     tpr=tf.divide(tf.cast(tp,tf.float32),tf.add(tp,fp))
#     tnr=tf.divide(tf.cast(tn,tf.float32),tf.add(tn,fn))
#     loss=tf.reduce_sum(tf.constant(1.0)-tf.scalar_mul(0.5,tf.add(tpr,tnr)))
    
    
    optimizer = tf.train.AdamOptimizer(1e-5)
    #train_step=optimizer.minimize(loss)
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        train_op = optimizer.minimize(loss)
    
    

(?,)


In [16]:
from tensorflow.python import debug
from pyntcloud import PyntCloud



In [17]:
# from random import shuffle
# trix=[cc for cc in range(TRAIN_DATASET.__len__())]
# shuffle(trix)
# np.save('./Windows/trix', trix, allow_pickle=True)

In [18]:
trix = np.load('./Windows/trix.npy')

In [19]:
trix=list(trix)

In [20]:
import datetime
import logging
from random import shuffle
import pandas as pd
from tensorflow.python import debug as tf_debug
print(datetime.datetime.now().strftime("%a, %d %B %Y %I:%M:%S"))

IOU_list = []
point_cld = []
saver = tf.train.Saver()

prob = []
voxels_train=[]
center_label=[]
centroids=[]
centroids_unique=[]
corners=[]
lbh=[]
heading=[]
extras=[]
pedc=0
selwin=[]  
windows=[]
residuallbh=[]
for i in trix[:3*len(trix)//4]:
    #while pedc<6000:
        if TRAIN_DATASET.type_list[i]=='Pedestrian':
            pedc+=1
            print('processing example number: ',pedc)
            point_set,labels,a,b,c,d,e,f,cls = TRAIN_DATASET.__getitem__(i)
            point_set,labels,a,b,c,d,e,f,cls = TRAIN_DATASET.__getitem__(i)
            corners_gt = TRAIN_DATASET.get_center_view_box3d(i)
            #extras.append(TRAIN_DATASET.__getitem__(i))
            index = np.argsort(point_set[:,2])
            point_set= point_set[index]
            labels = labels[index]
            s5,e5,l = indices(1.0,0.1,point_set,labels, corners_gt)

            if s5.size:

                ind_5 = get_list(s5,e5, num_points=512)

                point_set = np.expand_dims(point_set, axis=0)
                wc=0
                for ci in range(len(l)):

                    if l[ci]:
                        while(wc<2):
                            wc+=1
                            #point_cld.append(point_set[0,ind_5[ci].reshape((-1)),:3])

                            seg = point_set[0,ind_5[ci].reshape((-1))]
                            #print(seg.shape)
                            point_cld.append(seg)
                            selwin.append(seg)
                            
#                             seg = pd.DataFrame(seg[:,:3])
#                             seg.columns = ['x','y','z']
#                             cloud = PyntCloud(seg)

#                             voxelgrid_id = cloud.add_structure("voxelgrid", size_x=0.05, size_y=0.05, size_z=0.05, regular_bounding_box=False)
#                             voxelgrid = cloud.structures[voxelgrid_id]
#                             voxels_train.append(voxelgrid)
                            
                            
                            #print(pedc, seg.drop_duplicates().shape[0])
                            #center_label.append(TRAIN_DATASET.get_center_view_box3d_center(i))
                            #centroids_unique.append(np.mean(np.unique(np.array(point_set[0,ind_list[:,0]]),axis=0), axis=0))
                            #centroids.append(np.mean(np.array(point_set[0,ind_list[:,0]]),axis=0))
                            #corners.append(corners_gt)
                            lbh.append(provider.class2size(d,e))
                            extras.append(TRAIN_DATASET.__getitem__(i))
                            #lbh.append(TRAIN_DATASET.__getitem__(i)[6])
                            #cloud.plot()





Mon, 11 June 2018 02:17:34
processing example number:  1
processing example number:  2
processing example number:  3
processing example number:  4
processing example number:  5
processing example number:  6
processing example number:  7
processing example number:  8


KeyboardInterrupt: 

In [ ]:
np.array(extras).shape

In [ ]:
np.save('./Windows/selwin',np.array(selwin), allow_pickle=True)

np.save('./Windows/selextras',np.array(extras) , allow_pickle=True)

In [ ]:
# import numpy as np
# lbh=np.load('./Windows/lbh_jtrue2.npy', allow_pickle=True)
# voxels_train=np.load('./Windows/voxels_jtrue2.npy', allow_pickle=True)

In [ ]:
import datetime
import logging
from random import shuffle
import pandas as pd
#logging.basicConfig(filename='run_1.log',level=logging.DEBUG)
from tensorflow.python import debug as tf_debug
print(datetime.datetime.now().strftime("%a, %d %B %Y %I:%M:%S"))

IOU_list = []
point_cld = []
saver = tf.train.Saver()

prob = []
voxels_train=[]
center_label=[]
centroids=[]
centroids_unique=[]
corners=[]
lbh=[]
heading=[]
extras=[]
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    #sess=tf_debug.LocalCLIDebugWrapperSession(sess)
    sess.run(tf.global_variables_initializer())
    variables_names = [v.name for v in tf.trainable_variables()]
    values = sess.run(variables_names)
    #for k, v in zip(variables_names, values):
        #print("Variable: ", k)
        #print("Shape: ", v.shape)
    # uncomment this when we want to load the checkpoint model
    saver.restore(sess, "./checkpoints/model_new_labels_pointwiseIOU_5_10000.ckpt")
    for j in range(6,7):   
        total_tp=0
        total_fp=0
        total_tn=0
        total_fn=0
        total_tpnums = 0
        total_fpnums = 0
        total_fnnums = 0
        
        
        #print(datetime.datetime.now().strftime("%a, %d %B %Y %I:%M:%S"))
        
        #print('Epoch Number........................................',j)
        pedc=0
        
        for i in trix:
        #todo reformat code to have lists instead of many variable
            if TRAIN_DATASET.type_list[i]=='Pedestrian':
                pedc+=1
                if pedc%500==0:
                    total_tp=0
                    total_fp=0
                    total_tn=0
                    total_fn=0
                    total_tpnums = 0
                    total_fpnums = 0
                    total_fnnums = 0
                point_set,labels,a,b,c,d,e,f,cls = TRAIN_DATASET.__getitem__(i)
                #print(point_set.shape)
                lbh.append(provider.class2size(d,e))
                heading.append(provider.class2angle(b,c,12))
                extras.append(TRAIN_DATASET.__getitem__(i))
                index = np.argsort(point_set[:,2])
                #raw_input()

                point_set= point_set[index]
                #print(point_set[:25])
                labels = labels[index]

                corners_gt = TRAIN_DATASET.get_center_view_box3d(i)
                
                #print(point_set.shape)

                s5,e5,l = indices(1.0,0.1,point_set,labels, corners_gt)
                
                for u in range(len(s5)):
                        aa = np.max(np.array(point_set[np.arange(s5[u], e5[u])]), axis=0)-np.min(np.array(point_set[np.arange(s5[u], e5[u])]), axis=0)
                        #print(aa)
                        #print('hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh')
                        #print(aa)
                        #if aa[2]>1.05:
                        #    print('$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')
                        #    print(aa)
                
                
                #l1 = (l1>=0.6)
                
                #print('Label shape', l.shape)

#                 s1, e1, l1 = indices(i, 1.1, 0.2)

#                 s2, e2, l2 = indices(i, 1.3, 0.2)

#                 s3, e3, l3 = indices(i, 0.7, 0.1)

#                 s4, e4, l4 = indices(i, 0.5, 0.1)
                
                if s5.size :#and s2.size and s3.size and s4.size and s1.size:

                    ind_5 = get_list(s5,e5, num_points=512)
                    #print(ind_5.shape)
                    for u in range(len(ind_5)):
                        aa = np.max(np.array(point_set[ind_5[u]]), axis=0)-np.min(np.array(point_set[ind_5[u]]), axis=0)
                        #print(aa)
                        if aa[0,2]>1.05:
                            print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')
                            print(aa)
                            
                            

#                     ind_1 = get_list(s1,e1, num_points=512)

#                     ind_2 = get_list(s2,e2, num_points=512)

#                     ind_3 = get_list(s3,e3, num_points=512)

#                     ind_4 = get_list(s4,e4, num_points=512)
#                     t_windows=s1.shape[0]+s2.shape[0]+s3.shape[0]+s4.shape[0]+s5.shape[0]

                    point_set = np.expand_dims(point_set, axis=0)
                        
                    x,tpos,fpos,tneg,fneg, tpnums, maxind =sess.run([ loss,tp,fp,tn,fn, tp_num, maxi],feed_dict={idx_5:ind_5,\
                                                               label_out_5: l.astype(np.int32),\
                                                               inputs:point_set, is_training: True})
                    
                    #print(point_set.shape)
                    ind_list = ind_5[maxind]
                    #print(np.max(ind_list)-np.min(ind_list))
                    #print(ind_list[0])
                    #print(ind_list.shape)
                    seg = point_set[0,ind_list[:,0]]
                    #print(seg.shape)
                    point_cld.append(seg)
                    
                    seg = pd.DataFrame(seg[:,:3])
                    seg.columns = ['x','y','z']
                    cloud = PyntCloud(seg)
                    cloud.plot()
                    voxelgrid_id = cloud.add_structure("voxelgrid", size_x=0.05, size_y=0.05, size_z=0.05, regular_bounding_box=False)
                    voxelgrid = cloud.structures[voxelgrid_id]
                    voxels_train.append(voxelgrid)
                    print(pedc, seg.drop_duplicates().shape[0])
                    center_label.append(TRAIN_DATASET.get_center_view_box3d_center(i))
                    centroids_unique.append(np.mean(np.unique(np.array(point_set[0,ind_list[:,0]]),axis=0), axis=0))
                    centroids.append(np.mean(np.array(point_set[0,ind_list[:,0]]),axis=0))
                    corners.append(corners_gt)
                    
                    #heading angle
                    
                    #

In [ ]:
# voxels_train_np=[i.get_feature_vector() for i in voxels_train]

In [ ]:
obj_voxels=[]
#obj_centroids=[]
#obj_centroids_unique=[]
#obj_center_label=[]
#obj_corners=[]
obj_lbh=[]
obj_extras=[]
#obj_extras=[]
#obj_heading=[]
for i in range(len(voxels_train)):
    try :
        obj_voxels.append(voxels_train[i].get_feature_vector(mode='density'))
        #obj_centroids.append(centroids[i])
        #obj_centroids_unique.append(centroids_unique[i])
        #obj_center_label.append(center_label[i])
        #obj_corners.append(corners[i])
        obj_lbh.append(lbh[i])
        obj_extras.append(extras[i])
        #obj_extras.append(extras[i])
        #obj_heading.append(heading[i])
    except IndexError:
        pass
    except ValueError:
        pass

In [ ]:
np.save('./Windows/voxels_denstrue2',np.array(obj_voxels), allow_pickle=True)

np.save('./Windows/lbh_denstrue2',np.array(obj_lbh) , allow_pickle=True)

In [ ]:

lbh=np.load('./Windows/lbh_jtrue2.npy', allow_pickle=True)
voxels_train=np.load('./Windows/voxels_jtrue2.npy', allow_pickle=True)

In [ ]:
del voxels_train
del lbh

In [ ]:
len(obj_voxels),len(obj_centroids),len(obj_centroids_unique),len(obj_center_label),len(obj_corners),len(obj_lbh),len(obj_extras),len(obj_heading)

In [ ]:
#check the distances

import matplotlib.pyplot as plt
print(np.sqrt(np.sum((np.array(obj_center_label)-np.array(obj_centroids)[:,:3])**2, axis=1)))
print(np.sqrt(np.sum((np.array(obj_center_label)-np.array(obj_centroids)[:,:3])**2, axis=1)))
print(np.mean(np.sqrt(np.sum((np.array(obj_center_label)-np.array(obj_centroids)[:,:3])**2, axis=1))))
plt.hist(np.sqrt(np.sum((np.array(obj_center_label)-np.array(obj_centroids)[:,:3])**2, axis=1)), bins=100)

In [ ]:
#check the distances

import matplotlib.pyplot as plt
print(np.sqrt(np.sum((np.array(obj_center_label)-np.array(obj_centroids_unique)[:,:3])**2, axis=1)))
print(np.sqrt(np.sum((np.array(obj_center_label)-np.array(obj_centroids_unique)[:,:3])**2, axis=1)))
print(np.mean(np.sqrt(np.sum((np.array(obj_center_label)-np.array(obj_centroids_unique)[:,:3])**2, axis=1))))
plt.hist(np.sqrt(np.sum((np.array(obj_center_label)-np.array(obj_centroids_unique)[:,:3])**2, axis=1)), bins=20)

In [ ]:
# for i in range(len(voxels_train)):
#     if i not in [289,315,431,728,746,1283,1543,1865,2128,3485,3753,4429,4928,5048,5432,5907,6191,6999,7100,7574,\
#                 7687,9404,10179]:
#         voxels_train[i].get_feature_vector()

In [ ]:
# obj_voxels = []
# center_label = []
# centroids= []
# print(np.array(point_cld[0]).shape)
# for i in range(len(voxels_train)):
#     if i not in [289,315,431,728,746,1283,1543,1865,2128,3485,3753,4429,4928,5048,5432,5907,6191,6999,7100,7574,\
#                 7687,9404,10179]:
#         obj_voxels.append(voxels_train[i].get_feature_vector())
#         #center_label.append(TRAIN_DATASET.get_center_view_box3d_center(trix[i]))
#         #centroids.append(np.mean(np.unique(np.array(point_cld[i]),axis=0), axis=0))

In [ ]:
obj_voxels_modified = []
#center_label_modified = []
#centroids_modified = []
#obj_corners_modified=[]
obj_lbh_modified=[]
#obj_extras_modified=[]
#obj_heading_modified=[]

for i in range(len(obj_voxels)):
    dist = np.sqrt(np.sum((np.array(obj_center_label[i])-np.array(obj_centroids)[i:i+1,:3])**2, axis=1))
    if dist<1:
        #print(dist)
        obj_voxels_modified.append(obj_voxels[i])
        center_label_modified.append(obj_center_label[i])
        centroids_modified.append(obj_centroids[i])
        obj_corners_modified.append(obj_corners[i])
        obj_lbh_modified.append(obj_lbh[i])
        obj_extras_modified.append(obj_extras[i])
        obj_heading_modified.append(obj_heading[i])

In [ ]:
len(obj_voxels_modified),len(center_label_modified),len(centroids_modified),len(obj_corners_modified),len(obj_lbh_modified),len(obj_extras_modified),len(obj_heading_modified)

In [ ]:
max_x,max_y,max_z=0,0,0
for v in obj_voxels:
    max_x=max(max_x,v.shape[0])
    max_y=max(max_y,v.shape[1])
    max_z=max(max_z,v.shape[2])
print( max_x,max_y,max_z)

In [ ]:
pad_shape=[30,45,20]
padded_voxels=[]
ccc=0
for v in obj_voxels:
    ccc+=1
    print(ccc)
    dx=pad_shape[0]-v.shape[0]
    dy=pad_shape[1]-v.shape[1]
    dz=pad_shape[2]-v.shape[2]
    npad=((dx//2,dx-dx//2),(dy//2,dy-dy//2),(dz//2,dz-dz//2))
    padded_voxels.append(np.pad(v,pad_width=npad,mode='constant',constant_values=0))

In [ ]:
np.save('./Windows/padded_voxels_densetrue',np.array(padded_voxels), allow_pickle=True)
np.save('./Windows/fulllbh_densetrue',np.array(obj_lbh) , allow_pickle=True)
np.save('./Windows/extras_densetrue',np.array(obj_extras) , allow_pickle=True)



In [ ]:
v=np.load('./Windows/lbh_densetrue.npy', allow_pickle=True)
v=np.load('./Windows/padded_voxels_densetrue.npy', allow_pickle=True)
v=np.load('./Windows/extras_densetrue.npy', allow_pickle=True)

In [ ]:
obj_corners_modified[0]

In [ ]:
obj_corners_modified[0][:,1].max()

In [ ]:
list(np.unique(obj_corners_modified[0][:,1],1)[1])

In [ ]:
testi=[]
for i in range(100):
    if TRAIN_DATASET.__getitem__(trix[i])[-1][1]:
        testi.append(i) 

In [ ]:
TRAIN_DATASET.__getitem__(trix[4])[5:7]

In [ ]:
for i in testi:
    print(provider.class2size(TRAIN_DATASET.__getitem__(trix[i])[5],TRAIN_DATASET.__getitem__(trix[i])[6]))

In [ ]:
TRAIN_DATASET.get_center_view_box3d(trix[4])

In [ ]:
obj_corners_modified[0]

In [ ]:
for c in obj_corners_modified[:len(testi)]:
    print(finddim(c))

In [ ]:
def finddim(corners):
    c=np.copy(corners)
    height=c[:,1].max()-c[:,1].min()
    fourp=np.unique(np.delete(c,1,1),axis=0)
    point1=fourp[0]
    d=[]
    for point2 in fourp[1:]:
        d.append(np.linalg.norm(point1-point2))
    d=sorted(d)
    breadth=d[0]
    length=d[1]
    return np.array([length,breadth,height])
lbh=[]
for c in obj_corners_modified:
    lbh.append(finddim(c))

In [ ]:
residual_centers=[]

for i in range(len(centroids_modified)):
    residual_centers.append(center_label_modified[i]-np.array(centroids_modified)[i,:3])
    

In [ ]:
for residue in residual_centers:
    if np.sqrt(np.sum(residue**2))>1:
        print('ERROR')

In [ ]:
np.save('./Windows/centers',center_label_modified, allow_pickle=True)
np.save('./Windows/centroids',centroids_modified , allow_pickle=True)
np.save('./Windows/corners',obj_corners_modified, allow_pickle=True)
np.save('./Windows/lbh',obj_lbh_modified, allow_pickle=True)
np.save('./Windows/extras',obj_extras_modified, allow_pickle=True)
np.save('./Windows/heading_angle',obj_heading_modified, allow_pickle=True)
np.save('./Windows/res_centerlabels', residual_centers, allow_pickle=True)
np.save('./Windows/voxels', padded_voxels, allow_pickle=True)

In [ ]:
plt.hist(rdist,5)

In [ ]:
# np.save('./Windows/voxels', padded_voxels, allow_pickle=True)
# np.save('./Windows/centerlabels', residual_centers, allow_pickle=True)
# np.save('./Windows/lbh', lbh, allow_pickle=True)
# np.save('./Windows/corners', obj_corners_modified, allow_pickle=True)

In [ ]:
for v in padded_voxels:
    print(v.shape)

In [ ]:
#np.linalg.norm(np.array(center_label[:1]), np.array(centroids)[:1,:3], axis=1)
print(center_label[:1])
print(np.array(centroids)[:1,:3])
print(np.sum((np.array(center_label[:1])-np.array(centroids)[:1,:3])**2))

In [ ]:
a,b,c,d,e,f,g,h,i = TRAIN_DATASET.__getitem__(100)
print(c)
print(TRAIN_DATASET.get_center_view_box3d_center(100))

In [ ]:
np.unique(point_cld[i],axis=0)

In [ ]:
rnge= []
for i in range(len(point_cld)):
    rnge.append(np.max(np.array(point_cld[i]), axis=0)-np.min(np.array(point_cld[i]), axis=0))
print(len(rnge))

In [ ]:
np.max(np.array(rnge), axis=0)

In [ ]:
np.min(np.array(point_cld[0])[:,2])

In [ ]:
np.max(np.array(point_cld[0]), axis=0)

In [ ]:
outliers = []
for i in range(len(point_cld)):
    o = np.max(np.array(point_cld[i])[:,2], axis=0)-np.min(np.array(point_cld[i])[:,2], axis=0)
    if o>1.0:
        outliers.append([i,o])
print(len(outliers))

In [ ]:
ex = 10
print(voxels_train[ex].get_feature_vector().shape)
plotFromVoxels(voxels_train[ex].get_feature_vector())

In [ ]:
j= 0
for i in range(1000):
    #print(TRAIN_DATASET.type_list[trix[i]])
    if TRAIN_DATASET.type_list[trix[i]]=='Pedestrian':
        if j==60:
            print(j)
            print(i)
            seg,a,b,c,d,e,f,g,h = TRAIN_DATASET[60]     
            seg = pd.DataFrame(seg[:,:3])
            seg.columns = ['x','y','z']
            cloud = PyntCloud(seg)
            cloud.plot()
                
            
        j += 1
ex  = 60
seg,a,b,c,d,e,f,g,h = TRAIN_DATASET[trix[386]]
print(seg.shape)                    
seg = pd.DataFrame(seg[:,:3])
seg.columns = ['x','y','z']
cloud = PyntCloud(seg)
cloud.plot()

In [ ]:
import sys
import os

import scipy.ndimage as nd
import scipy.io as io
import numpy as np
import matplotlib.pyplot as plt
import skimage.measure as sk

from mpl_toolkits import mplot3d

import trimesh
from stl import mesh


In [ ]:
def plotFromVF(vertices, faces):
    input_vec = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
    for i, f in enumerate(faces):
        for j in range(3):
            input_vec.vectors[i][j] = vertices[f[j],:]
    figure = plt.figure()
    axes = mplot3d.Axes3D(figure)
    axes.add_collection3d(mplot3d.art3d.Poly3DCollection(input_vec.vectors))
    scale = input_vec.points.flatten(-1)
    axes.auto_scale_xyz(scale, scale, scale)
    plt.show()

def plotFromVoxels(voxels):
    #print(voxels.nonzero())
    x,y,z = voxels.nonzero()
    fig = plt.figure()
    #ax = fig.add_subplot(111, projection='3d')
    #ax.scatter( 0.05*x, z*0.05, -0.1*y, zdir='z', c= 'red', norm=0.05)
    ax = fig.gca(projection='3d')
    ax.voxels(0.05*voxels)
    plt.show()

def getVFByMarchingCubes(voxels, threshold=0.5):
    #print(len(sk.marching_cubes(voxels, level=threshold)))
    v, f, n, m =  sk.marching_cubes(voxels, level=threshold)
    return v, f

def plotMeshFromVoxels(voxels, threshold=0.5):
    #print(getVFByMarchingCubes(voxels, threshold))
    v,f = getVFByMarchingCubes(voxels, threshold)
    plotFromVF(v,f)

def plotVoxelVisdom(voxels, visdom, title):
    v, f = getVFByMarchingCubes(voxels)
    visdom.mesh(X=v, Y=f, opts=dict(opacity=0.5, title=title))

In [ ]:
#plotFromVoxels(np.array([[[0,1,0],[0,0,0],[0,1,1]],[[1,1,1],[1,1,1],[0,1,1]]]))
plotMeshFromVoxels(np.array([[[0,1,0],[0,0,0],[0,1,1]],[[1,1,1],[1,1,1],[0,1,1]]]))

In [ ]:
voxelgrid_id = cloud.add_structure("voxelgrid", size_x=0.05, size_y=0.05, size_z=0.05)

In [ ]:
seg['z'].max()-seg['z'].min()

In [ ]:
voxelgrid = cloud.structures[voxelgrid_id]

In [ ]:
voxelgrid.get_feature_vector().shape

In [ ]:
seg.drop_duplicates()

In [ ]:
cloud.plot()

In [ ]:
seg.isnull().any()

In [ ]:
def pad_trunc_voxels():
    for voxel in voxels_train:
        dim=voxel.shape
        temp=np.zeros((30,40,20))
        temp
            

In [ ]:
print(voxelgrid.get_feature_vector().shape)
print(np.sum(voxelgrid.get_feature_vector()))
print(voxelgrid.get_feature_vector())

In [ ]:
voxelgrid.plot()

In [ ]:
cloud.plot()

In [ ]:
import datetime
import logging
from random import shuffle
#logging.basicConfig(filename='run_1.log',level=logging.DEBUG)
from tensorflow.python import debug as tf_debug
print(datetime.datetime.now().strftime("%a, %d %B %Y %I:%M:%S"))

IOU_list = []
saver = tf.train.Saver()
trix=[cc for cc in range(TRAIN_DATASET.__len__())]
shuffle(trix)
prob = []
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    #sess=tf_debug.LocalCLIDebugWrapperSession(sess)
    sess.run(tf.global_variables_initializer())
    variables_names = [v.name for v in tf.trainable_variables()]
    values = sess.run(variables_names)
    for k, v in zip(variables_names, values):
        print("Variable: ", k)
        print("Shape: ", v.shape)
    # uncomment this when we want to load the checkpoint model
    saver.restore(sess, "./checkpoints/model_new_labels_pointwiseIOU_5_10000.ckpt")

    
    for j in range(6,15):   
        total_tp=0
        total_fp=0
        total_tn=0
        total_fn=0
        total_tpnums = 0
        total_fpnums = 0
        total_fnnums = 0
        
        
        print(datetime.datetime.now().strftime("%a, %d %B %Y %I:%M:%S"))
        
        print('Epoch Number........................................',j)
        pedc=0
        
        for i in trix:
        #todo reformat code to have lists instead of many variable
            if TRAIN_DATASET.type_list[i]=='Pedestrian':
                pedc+=1
                if pedc%500==0:
                    total_tp=0
                    total_fp=0
                    total_tn=0
                    total_fn=0
                    total_tpnums = 0
                    total_fpnums = 0
                    total_fnnums = 0
                point_set,labels,a,b,c,d,e,f,cls = TRAIN_DATASET.__getitem__(i)

                index = np.argsort(point_set[:,2])
                point_set = point_set[index]
                labels = labels[index]

                corners_gt = TRAIN_DATASET.get_center_view_box3d(i)
                
 
                s5,e5,l = indices(1.0,0.1,point_set,labels, corners_gt)
        
                #l1 = (l1>=0.6)
                
                print('Label shape', l.shape)


                if s5.size :#and s2.size and s3.size and s4.size and s1.size:

                    ind_5 = get_list(s5,e5, num_points=512)



                    point_set = np.expand_dims(point_set, axis=0)
                        
                    _, x,tpos,fpos,tneg,fneg, tpnums, maxind =sess.run([train_op, loss,tp,fp,tn,fn, tp_num, maxi],feed_dict={idx_5:ind_5,\
                                                               label_out_5: l.astype(np.int32),\
                                                               inputs:point_set, is_training: True})
                    total_tp+=tpos
                    total_fp+=fpos
                    total_tn+=tneg
                    total_fn+=fneg
                    total_tpnums+= tpnums
                    total_fpnums+= (1-tpnums)
                    total_fnnums+= (1-tpnums)
                    ind_5[maxind]
                    print('----------------------------------------------------------------------------------')
                    print('Epoch ', j , 'Loss for the current example ',i,' = ', x,'pedc = ',pedc)
                    #logging.info('Loss for the current example ',i,' = ', x)
                    print('Metrics for current example: TP=',tpos,' FP=',fpos,' TN=',tneg,' FN=',fneg)
                    #logging.info('Metrics for current example: TP=',tpos,' FP=',fpos,' TN=',tneg,' FN=',fneg)  # will not print anything
                    #print('Cumulative metrics: TP=',total_tp,' FP=',total_fp,' TN=',total_tn,' FN=',total_fn)
                    print('Cumulative metrics: Precision = ',(total_tp*1.0)/(total_tp+total_fp),\
                          ' Recall = ',(total_tp*1.0)/(total_tp+total_fn))
                    #logging.info('Cumulative metrics: Precision = ',(total_tp*1.0)/(total_tp+total_fp),\
                    #      ' Recall = ',(total_tp*1.0)/(total_tp+total_fn))
                    print('Single metrics: Precision = ',(total_tpnums*1.0)/(total_tpnums+total_fpnums),\
                          ' Recall = ',(total_tpnums*1.0)/(total_tpnums+total_fnnums))
                    print('----------------------------------------------------------------------------------')
                    
                    if(pedc%5000==0):
                        save_path = saver.save(sess, "./checkpoints/model_new_labels_pointwiseIOU_"+str(j)+'_'+str(pedc)+".ckpt")
                    

                #print(c)
                #print(n)
            #i += 1
        #-----------------------------------------------------------------------------------------------#
        print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')
        k = 0
        Vtotal_tp=0
        Vtotal_fp=0
        Vtotal_tn=0
        Vtotal_fn=0
        Vtotal_tpnums = 0
        Vtotal_fpnums = 0
        Vtotal_fnnums = 0
        while k<len(VAL_DATASET):
        #todo reformat code to have lists instead of many variable

            if VAL_DATASET.type_list[k]=='Pedestrian':

                point_set,labels,a,b,c,d,e,f,cls = VAL_DATASET.__getitem__(k)

                index = np.argsort(point_set[:,2])
                point_set = point_set[index]
                labels = labels[index]

                
                corners_gt = VAL_DATASET.get_center_view_box3d(k)
                
                #print(point_set.shape)

                s5,e5,l = indices(1.0,0.1,point_set,labels, corners_gt)
                
                #l1 = l1>0.5

#                 s1, e1, l1 = indices(k, 1.1, 0.2, 'val')

#                 s2, e2, l2 = indices(k, 1.3, 0.2, 'val')

#                 s3, e3, l3 = indices(k, 0.7, 0.1, 'val')

#                 s4, e4, l4 = indices(k, 0.5, 0.1, 'val')

                if s5.size :#and s2.size and s3.size and s4.size and s1.size:

                    ind_5 = get_list(s5,e5, num_points=512)

#                     ind_1 = get_list(s1,e1, num_points=512)

#                     ind_2 = get_list(s2,e2, num_points=512)

#                     ind_3 = get_list(s3,e3, num_points=512)

#                     ind_4 = get_list(s4,e4, num_points=512)

#                     t_windows=s1.shape[0]+s2.shape[0]+s3.shape[0]+s4.shape[0]+s5.shape[0]

                    point_set = np.expand_dims(point_set, axis=0)
                    print(Vtotal_tp)
                    x,tpos,fpos,tneg,fneg, tp_nums, maxind =sess.run([loss,tp,fp,tn,fn, tp_num, maxi],feed_dict={idx_5:ind_5, \
                                                               label_out_5: l.astype(np.int32), inputs:point_set, is_training: True})


                    #window_points = ind_5[maxind]

                    #window_points = point_set[0,ind_5[maxind].reshape((-1))][:,:3]
                    #min_x, min_y, min_z = np.min(window_points,axis=0)
                    #max_x, max_y, max_z = np.max(window_points,axis=0)
                    #corners = [[min_x,max_y,max_z],[min_x,max_y,min_z],[max_x,max_y,min_z],[max_x,max_y,max_z],\
                    #           [min_x,min_y,max_z],[min_x,min_y,min_z],[max_x,min_y,min_z],[max_x,min_y,max_z]]
                    #corners=np.array(corners)
                    #corners_gt = VAL_DATASET.get_center_view_box3d(k)
                    #IOU, IOU_2D, inter, inter_vol = box_util.box3d_iou(corners, corners_gt)
                    #print('IOUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUU')
                    #print(IOU, IOU_2D, inter, inter_vol)
                    #IOU_list.append([IOU, IOU_2D, inter, inter_vol, k])
                    #if inter>0.4:
                    #    c += 1
                    #    if inter>0.5:
                    #        c1 += 1
                    #if inter_vol>0.4:
                    #    c2 += 1
                    #    if inter_vol>0.5:
                    #        c3 +=1 

                    print(Vtotal_tp)
                    Vtotal_tp +=tpos
                    Vtotal_fp +=fpos
                    Vtotal_tn +=tneg
                    Vtotal_fn +=fneg
                    Vtotal_tpnums+= tp_nums
                    Vtotal_fpnums+= (1-tp_nums)
                    Vtotal_fnnums+= (1-tp_nums)
                    print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')
                    print('Epoch ', j , 'Loss for the current example ',k,' = ', x)
                    #logging.info('Loss for the current example ',i,' = ', x)
                    print('Metrics for current example: TP=',tpos,' FP=',fpos,' TN=',tneg,' FN=',fneg)
                    #logging.info('Metrics for current example: TP=',tpos,' FP=',fpos,' TN=',tneg,' FN=',fneg)  # will not print anything
                    #print('Cumulative metrics: TP=',total_tp,' FP=',total_fp,' TN=',total_tn,' FN=',total_fn)
                    print('Cumulative metrics: Precision = ',(Vtotal_tp*1.0)/(Vtotal_tp+Vtotal_fp),\
                          ' Recall = ',(Vtotal_tp*1.0)/(Vtotal_tp+Vtotal_fn))

                    print('Single metrics: Precision = ',(Vtotal_tpnums*1.0)/(Vtotal_tpnums+Vtotal_fpnums),\
                          ' Recall = ',(Vtotal_tpnums*1.0)/(Vtotal_tpnums+Vtotal_fnnums))
                    #logging.info('Cumulative metrics: Precision = ',(Vtotal_tp*1.0)/(Vtotal_tp+Vtotal_fp),\
                    #      ' Recall = ',(Vtotal_tp*1.0)/(Vtotal_tp+Vtotal_fn))
                    print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')

            k += 1

        #----------------------------------------------------------------------------------------------#
#         prob = sess.run([sig],feed_dict={idx_5:ind_5, idx_1:ind_1, idx_2:ind_2, idx_3:ind_3, idx_4:ind_4,\
#                                                        label_out_5: l5.astype(np.int32), label_out_1: l1.astype(np.int32),\
#                                                        label_out_2: l2.astype(np.int32), label_out_3: l3.astype(np.int32),\
#                                                        label_out_4: l4.astype(np.int32), inputs:point_set,  is_training: False})
#         print(prob)
        print(ind_5)
        print(label_out_5)
        prob = sess.run([sig],feed_dict={idx_5:ind_5,  \
                                                       label_out_5: l.astype(np.int32), inputs:point_set,  is_training: True})
        print(prob)
print(datetime.datetime.now().strftime("%a, %d %B %Y %I:%M:%S") )

In [ ]:
n = np.random.rand(31,26,12)
padded_n=np.zeros((40,30,20))
padded_n[:min(n.shape[0],40),:min(n.shape[1],30),:min(n.shape[2],20)]=n


In [ ]:
inp=tf.placeholder(tf.float32,shhape=(1,30,45,20))

conv3d1=tf_util.conv3d(inp,num_output_channels=64,kernel_size=[3,3,3],scope='convr1', is_training = True)
conv3d2=tf_util.conv3d(conv3d1,num_output_channels=128,kernel_size=[3,3,3],scope='convr2', is_training = True)
conv3d3=tf_util.conv3d(conv3d2,num_output_channels=256,kernel_size=[3,3,3],scope='convr3', is_training = True)

dense=conv2d(conv3d2,num_output_channels=30,kernel_size=[40,40,30],scope='dense1r', is_training = True)
out=conv2d(dense,num_output_channels=2,kernel_size=[1,1,30],scope='dense2r', is_training = True)

In [ ]:
import pyntcloud
import pickle
import tensorflow as tf
import numpy as np
import importlib
from train import provider
from train import box_util

In [ ]:
import tensorflow as tf
import numpy as np
from model_util import huber_loss

In [ ]:
from model import tf_util

In [ ]:
centers=np.load('./Windows/centers.npy', allow_pickle=True)
centroids=np.load('./Windows/centroids.npy', allow_pickle=True)
corners=np.load('./Windows/corners.npy', allow_pickle=True)
lbh=np.load('./Windows/lbh.npy', allow_pickle=True)
extras=np.load('./Windows/extras.npy', allow_pickle=True)
heading_angle =np.load('./Windows/heading_angle.npy',allow_pickle=True)
residual_centers=np.load('./Windows/res_centerlabels.npy',  allow_pickle=True)
padded_voxels=np.load('./Windows/voxels.npy', allow_pickle=True)

In [ ]:
residual_lbh=np.array([x[6] for x in extras])
residual_angle=np.array([x[4] for x in extras])
angle_class=np.array([x[3] for x in extras])

In [ ]:
residual_angle[0]

In [ ]:
def conv_onehot_angle_bin(x):
    temp=np.zeros((12))
    temp[x]=1
    return temp
one_hot_angle_class=[conv_onehot_angle_bin(x) for x in angle_class]
residual_angle_labels=[one_hot_angle_class[i]*residual_angle[i] for i in range(residual_angle.shape[0])]

In [ ]:
one_hot_angle_class[0]

In [ ]:
residual_angle_labels[0]

In [ ]:
residual_centers[0]

In [ ]:
np.mean(np.linalg.norm(residual_lbh,axis=1))

In [ ]:
#def create_lbl(i):
#    temp=np.zeros(30)
#    temp[:]
    
targets = np.zeros((len(residual_angle_labels),30))
targets[:,:3] = residual_centers
targets[:,3:6] = residual_lbh
targets[:,6:18] = one_hot_angle_class
targets[:,18:30] = residual_angle_labels

In [ ]:
def boundingBoxRegression(inp):
    conv3d1=tf_util.conv3d(inp,num_output_channels=16,kernel_size=[2,2,2], scope='convr1', bn=True, is_training = True, bn_decay=True)
    #conv3d1 = tf_util.max_pool3d(conv3d1, [2,2,2], scope='maxpool1', stride= [2,2,2])
    conv3d11=tf_util.conv3d(inp,num_output_channels=16,kernel_size=[1,2,1], scope='convr11', bn=True, is_training = True, bn_decay=True)
    conv3d1o=tf.concat([conv3d1,conv3d11 ],axis=4)
    conv3d11 = tf_util.max_pool3d(conv3d1o, [2,2,2], scope='maxpool1', stride= [2,2,2])
    
    conv3d2=tf_util.conv3d(conv3d1,num_output_channels=128,kernel_size=[3,3,3], scope='convr2', bn=True, is_training = True, bn_decay=True)
    #conv3d2 = tf_util.max_pool3d(conv3d2, [2,2,2], scope='maxpool2', stride= [2,2,2])
    conv3d3=tf_util.conv3d(conv3d2,num_output_channels=256,kernel_size=[2,3,2], scope='convr3', bn=True, is_training = True, bn_decay=True)
    
    #deconv3d1=tf_util.conv2d_transpose(conv3d1,128,kernel_size=[1,1])
    #deconv3d2=tf_util.conv2d_transpose(conv3d2,128,)
    
    
    
    dense1 = tf.contrib.layers.flatten(conv3d3)
    dense2 = tf_util.fully_connected(dense1, 64, scope='denser2', bn=True, is_training=True, bn_decay=True)
    #dense3 = tf_util.fully_connected(dense2,128 , scope='denser3', bn=True, is_training=True, bn_decay=True)
    dense4 = tf_util.fully_connected(dense2, 30, activation_fn=None, scope='denser4')
    
    
    return dense4,conv3d1o

In [ ]:
from model_util import g_type_mean_size

In [ ]:
targets.shape

In [ ]:
tf.reset_default_graph()

with tf.device('/device:GPU:0'):

    voxel_inputs =tf.placeholder(tf.float32,[None,30,45,20, 1])#put placeholder

    regress_labels =tf.placeholder(tf.float32,[None,30])

    out,abc = boundingBoxRegression(voxel_inputs)
    abc=abc.get_shape()
    #loss = tf.losses.mean_squared_error(regress_labels, out, scope='loss')
    center_loss1=tf.norm(out-regress_labels,axis=1)#out[0:3]
    center_loss=tf.reduce_mean(center_loss1)
    center_hloss=huber_loss(center_loss1,delta=1.0)
    
    
    residual_sizes_labels=regress_labels[:,3:6]
    
    #residual_size_labels_normalized=tf.div(residual_sizes_labels,tf.constant(g_type_mean_size['Pedestrian'],dtype=tf.float32))
    size_loss1=tf.norm(out[:,3:6]-residual_sizes_labels, axis=1)

    #size_loss1=tf.norm(out[:,3:6]-residual_size_labels_normalized, axis=1)
    #size_hloss=huber_loss(size_loss1,delta=1.0)
    size_loss=tf.reduce_mean(size_loss1)
    
    
    
    
    heading_class_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=regress_labels[:,6:18],logits=out[:,6:18]))
    
    heading_residual_label=regress_labels[:,18:30]
    heading_residual_normalized_label=heading_residual_label/(np.pi/12)
    heading_hloss = huber_loss(tf.reduce_sum(tf.norm(out[:,18:30]-heading_residual_normalized_label,axis=1)*tf.to_float(regress_labels[:,6:18]), axis=1), delta=1.0)
    ####loss = tf.reduce_mean(tf.sqrt(tf.reduce_sum(tf.pow(out-regress_labels,2), axis=1)))
    
                                           
    #corner loss
                                         
    # Volume loss
    
    loss=1*size_loss#+0*heading_hloss+0*heading_class_loss+0*center_hloss

    #loss=0*center_hloss+1*size_hloss+0*heading_hloss+0*heading_class_loss
    optimizer = tf.train.AdamOptimizer(1e-3)
    #train_step=optimizer.minimize(loss)
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    
    with tf.control_dependencies(update_ops):
        train_op = optimizer.minimize(loss)

In [ ]:
pred_center = {}
test_pred_center = {}
all_losses_train = {}
all_losses_test={}
saver = tf.train.Saver()
with tf.Session(config=tf.ConfigProto(allow_soft_placement=False, log_device_placement=True)) as sess:

    sess.run(tf.global_variables_initializer())
    #variables_names = [v.name for v in tf.trainable_variables()]
    #values = sess.run(variables_names)
    #for k, v in zip(variables_names, values):
        #print("Variable: ", k)
        #print("Shape: ", v.shape)
    # uncomment this when we want to load the checkpoint model
    #saver.restore(sess, "./checkpoints/regression_"+str(0)+'_'+str(4100)+".ckpt")
    vox = np.load('./Windows/voxels.npy')
    #labels = np.load('./Windows/centerlabels.npy')
    assert vox.shape[0]==targets.shape[0]
    assert vox[0].shape==(30,45,20)
    
    for epoch in range(40):
        print('**************************************************************************************')
        i=0
        batch_size=1
        pred_center[epoch] = []
        all_losses_train[epoch] = []
        while(i<(vox.shape[0]-2000)):
            _, l,s_loss,c_loss,a, b,hc_loss, pred = sess.run([train_op, loss, size_loss, center_loss,size_loss1,residual_sizes_labels, heading_class_loss, out], feed_dict={voxel_inputs:np.reshape(vox[i:i+batch_size], (batch_size, vox[i:i+batch_size].shape[1],\
                                                                                        vox[i:i+batch_size].shape[2], vox[i:i+batch_size].shape[3], 1)),\
                                                       regress_labels: np.reshape(targets[i:i+batch_size],(batch_size,-1))})
#             _, l,s_loss,c_loss,a, b,hc_loss, pred = sess.run([train_op, loss, size_loss, center_loss,size_loss1,residual_sizes_labels, heading_class_loss, out], feed_dict={voxel_inputs:np.reshape(vox[i:i+batch_size], (batch_size, vox[i:i+batch_size].shape[1],\
#                                                                                         vox[i:i+batch_size].shape[2], vox[i:i+batch_size].shape[3], 1)),\
#                                                        regress_labels: np.reshape(targets[i:i+batch_size],(batch_size,-1))})
            i+=batch_size
            #print('epoch=',epoch,'batch=',i,'loss=',l)
            #print('OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO')
            #print(a)
            #print(b)
            #print(pred[:,3:6])
            print(l)
            #print('OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO')
            pred_center[epoch].append([pred,i,l])
            all_losses_train[epoch].append([s_loss,c_loss,hc_loss])
            if i%100<5:
                save_path = saver.save(sess, "./checkpoints/regression_"+str(epoch)+'_'+str(i)+".ckpt")
                print('%%%%%%%%%%%%%%%%%%%%%%%%%%AverageLoss%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
                print(np.mean(np.array(all_losses_train[epoch]),axis=0))
                np.save('./Windows/train_pred_'+str(epoch), np.array(pred_center[epoch]), allow_pickle=True)
                np.save('./Windows/train_loss_'+str(epoch), np.array(all_losses_train[epoch]), allow_pickle=True)
                
                #print(np.mean(np.array(pred_center[epoch]), axis=0)[2])
        test=vox.shape[0]-2000
        test_pred_center[epoch]=[]
        all_losses_test[epoch] = []

        while(test<(vox.shape[0])):
            l,s_loss,c_loss,hc_loss, pred = sess.run([loss, size_loss, center_loss, heading_class_loss,out], feed_dict={voxel_inputs:np.reshape(vox[test:test+batch_size], (batch_size, vox[test:test+batch_size].shape[1],\
                                                                                        vox[test:test+batch_size].shape[2], vox[test:test+batch_size].shape[3], 1)),\
                                                       regress_labels: np.reshape(targets[test:test+batch_size],(batch_size,-1))})
            test+=batch_size
            print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')
            print('epoch=',epoch,'batch=',test,'loss=',l)
            test_pred_center[epoch].append([pred,test,l])
            all_losses_test[epoch].append([s_loss,c_loss,hc_loss])
            np.save('./Windows/test_pred_'+str(epoch), np.array(test_pred_center[epoch]), allow_pickle=True)
            np.save('./Windows/test_loss_'+str(epoch), np.array(all_losses_test[epoch]), allow_pickle=True)
            if test%50<5:
                print('%%%%%%%%%%%%%%%%%%%%%%%%%%AverageLoss%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
                print(np.mean(np.array(all_losses_test[epoch]),axis=0))

                #print([l,s_loss,c_loss,hc_loss])
                
                #print(np.mean(np.array(test_pred_center[epoch]), axis=0)[2])

In [ ]:
all_losses_train[0][0]

In [ ]:
pred_center = {}
test_pred_center = {}
saver = tf.train.Saver()
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:

    sess.run(tf.global_variables_initializer())
    #variables_names = [v.name for v in tf.trainable_variables()]
    #values = sess.run(variables_names)
    #for k, v in zip(variables_names, values):
        #print("Variable: ", k)
        #print("Shape: ", v.shape)
    # uncomment this when we want to load the checkpoint model
    saver.restore(sess, "./checkpoints/regression_"+str(15)+'_'+str(5000)+".ckpt")
    vox = np.load('./Windows/voxels.npy')
    labels = np.load('./Windows/lbh.npy')
    assert vox.shape[0]==labels.shape[0]
    assert vox[0].shape==(30,45,20)
    
    for epoch in range(40):
        print('**************************************************************************************')
        i=0
        batch_size=5
        pred_center[epoch] = []
        while(i<(vox.shape[0]-2000)):
            _, l, center = sess.run([train_op, loss, out], feed_dict={voxel_inputs:np.reshape(vox[i:i+batch_size], (batch_size, vox[i:i+batch_size].shape[1],\
                                                                                        vox[i:i+batch_size].shape[2], vox[i:i+batch_size].shape[3], 1)),\
                                                       regress_labels: np.reshape(labels[i:i+batch_size],(batch_size,-1))})
            i+=batch_size
            print('epoch=',epoch,'batch=',i,'loss=',l)
            pred_center[epoch].append([center,i,l])
            if i%1000==0:
                save_path = saver.save(sess, "./checkpoints/lbhregression_"+str(epoch)+'_'+str(i)+".ckpt")
                print('%%%%%%%%%%%%%%%%%%%%%%%%%%AverageLoss%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
                print(np.mean(np.array(pred_center[epoch]), axis=0)[2])
        test=vox.shape[0]-2000
        test_pred_center[epoch]=[]
        while(test<(vox.shape[0])):
            l, center = sess.run([loss, out], feed_dict={voxel_inputs:np.reshape(vox[test:test+batch_size], (batch_size, vox[test:test+batch_size].shape[1],\
                                                                                        vox[test:test+batch_size].shape[2], vox[test:test+batch_size].shape[3], 1)),\
                                                       regress_labels: np.reshape(labels[test:test+batch_size],(batch_size,-1))})
            test+=batch_size
            print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')
            print('epoch=',epoch,'batch=',test,'loss=',l)
            test_pred_center[epoch].append([center,test,l])
            if test%500<5:
                print('%%%%%%%%%%%%%%%%%%%%%%%%%%AverageLoss%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
                print(np.mean(np.array(test_pred_center[epoch]), axis=0)[2])

In [ ]:
#huber loss
#gradient clipping


In [ ]:
labels.shape

In [ ]:
plt.hist(labels[:,0])

In [ ]:
np.std(labels[:,0])

In [ ]:
plt.hist(labels[:,1])

In [ ]:
np.std(labels[:,1])

In [ ]:
plt.hist(labels[:,2])

In [ ]:
np.std(labels[:,2])